<a href="https://colab.research.google.com/github/pinilDissanayaka/Fine-Tuning-Pretrained-DistilBERT-sinhala-sentiment-analysis/blob/main/Fine_Tuning_Pretrained_DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets
! pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
pip install transformers

In [3]:
import pandas as pd
import numpy as np
from datasets import load_dataset, DatasetDict, Dataset
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TFTrainingArguments

In [15]:
huggingface_dataset=load_dataset("sinhala-nlp/sinhala-sentiment-analysis")

In [16]:
huggingface_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'article_id', 'title', 'author', 'date', 'body', 'comment_author', 'comment_date', 'comment_phrase', 'comment_sentiment'],
        num_rows: 7229
    })
    test: Dataset({
        features: ['id', 'article_id', 'title', 'author', 'date', 'body', 'comment_author', 'comment_date', 'comment_phrase', 'comment_sentiment'],
        num_rows: 1819
    })
})

In [17]:
huggingface_dataset["train"]

Dataset({
    features: ['id', 'article_id', 'title', 'author', 'date', 'body', 'comment_author', 'comment_date', 'comment_phrase', 'comment_sentiment'],
    num_rows: 7229
})

In [18]:
x_train=list(huggingface_dataset["train"]['comment_phrase'])
y_train=list(huggingface_dataset["train"]['comment_sentiment'])

In [19]:
x_test=list(huggingface_dataset["test"]['comment_phrase'])
y_test=list(huggingface_dataset["test"]['comment_sentiment'])

In [20]:
print(f"x_train: {len(x_train)}")
print(f"y_train: {len(y_train)}")
print(f"x_test: {len(x_test)}")
print(f"y_test: {len(y_test)}")

x_train: 7229
y_train: 7229
x_test: 1819
y_test: 1819


In [21]:
np.unique(y_train)

array(['NEGATIVE', 'NEUTRAL', 'POSITIVE'], dtype='<U8')

In [22]:
label_encoder=LabelEncoder()
y_train=label_encoder.fit_transform(y_train)
y_test=label_encoder.transform(y_test)

In [23]:
encorded_labels=list(np.unique(y_train))

decoded_labels=label_encoder.inverse_transform(encorded_labels)


for encoded_label, decoded_label in zip(encorded_labels, decoded_labels):
    print(f"Encoded Label: {encoded_label} --> Decoded Label: {decoded_label}")


num_labels=len(encorded_labels)

print(f"num_labels: {num_labels}")

Encoded Label: 0 --> Decoded Label: NEGATIVE
Encoded Label: 1 --> Decoded Label: NEUTRAL
Encoded Label: 2 --> Decoded Label: POSITIVE
num_labels: 3


In [24]:
d_bert_tokenizer=DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [25]:
x_train_tokenized=d_bert_tokenizer(x_train, truncation=True, padding=True)
x_test_tokenized=d_bert_tokenizer(x_test, truncation=True, padding=True)

In [26]:
train_df=pd.DataFrame(x_train_tokenized)

In [119]:
x_train_tokenized_dict=dict(x_train_tokenized)
x_train_tokenized_dict['labels']=y_train

x_test_tokenized_dict=dict(x_test_tokenized)
x_test_tokenized_dict['labels']=y_test

In [120]:
x_test_tokenized_dict.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [121]:
train_df=Dataset.from_dict(x_train_tokenized_dict)

test_df=Dataset.from_dict(x_test_tokenized_dict)

In [122]:
train_df

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 7229
})

In [123]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

In [128]:
d_bert_model=DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer = Trainer(
    model=d_bert_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_df,         # training dataset
    eval_dataset=test_df             # evaluation dataset
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args_tf.py:295: FutureWarning: The n_gpu argument is deprecated and will be removed in a future version, use n_replicas instead.
  warnings.warn(


In [ ]:
pip install